# Use Jinja and Frozen Flask to build high performance static website
### Tanay V. PrabhuDesai
### Nelkinda Software Craft Pvt. Ltd.


## What is this talk about
1. Metrics for high performance website
2. Existing blog generators
3. Jinja for writing blog pages
4. Using Frozen Flask
5. Rendering of HTML pages
6. Minification of CSS and JS
7. Compression of images
8. Parallelizing some of the tasks

## Metrics for high performance
### Automated improvements
- Minification of CSS and JS
- Combining CSS and JS into a single file
- Reducing the size of images
### Others
- Setting browser caching
- 

# Google Page Insights

```
https://developers.google.com/speed/pagespeed/insights/?url=somewebsite.com
```

### From this

TODO: add an image here


### To this

TODO: add an image here

## Existing static blog generators

#### The existing static blog generators are mostly driven by users who want to write blogs using Markdown

#### Jekyll is one of the most popular static blog generators, in which users have to write blogs in Markdown

#### Sometimes there is lack of flexibility when we are writing blogs, and Markdown starts becoming HTML

## Strengths and weaknesses of Markdown

#### Strengths:
- Easy to read when presented in plain text
- Not a lot of coding experience is needed
- Ideal for writing blog/book/document that will mostly be read in plain text

#### Weaknesses:
- Not possible for customization of the styl layout for different pages
- Have to reduce the customization by including a lot of HTML tags
- Not designed for writing something like a website that needs variation of styles 

## Using Jinja for front end web pages

- Compiles to HTML
- Provides template inheritance
- Increases code reusability
- Incresses writeability (can focus on writing just the content of that page)
- Helps passing Python objects to webpages to be rendered as HTML

## Frozen Flask for rendering static html

- Frozen Flask converts all the `GET` endpoints of your Flask application into static HTML files
- These endpoints look like normal endpoints that you will see in Flask projects

```python
    @app.route('/')
    def root():
        return template.render(css_file='main.css')
```
- Just keep in mind that these endpoints should be GET endpoints

## Serving static files through Frozen Flask
- Suppose you want to serve a static resource file from your server
- The path for one of the image file is: `/static/image/screenshot.png`
- There are many other images to be served on the same path
- The images are actually stored in a different directory in the actual path
- You cannot just write a code like this
```python
    @app.route('/res/images/<path>')
    def images(path):
        return send_from_directory('static/images/', path)
```

- Well, you can do that but it is not enough. You need something more
- You have to freeze endpoints in addition to just declaring the endpoints

## Freezing endpoints in Frozen Flask

- In order for the endpoints serving static files, we need register generators for those endpoints
- This needs to be done so that Frozen Flask knows for what files should the endpoint freeze
- The freezers are decorators applied on generators which generate dictionaries
```python
    @freezer.register_generator
    def images():
        image_paths = [image_path for image_path in generate_all_image_path(IMAGE_PATH)]
        for image_path in image_paths:
            yield {'path': image_path}
```
- Key of the dictionary is the same variable specified in the endpoint

## Complex and useful freezing

- If you need an endpoint to follow the traditional blog structure
- With a path simillar to `somewebsite/blog/2017/11/04/some-post.html`
- The Jinja template written is at the location `posts/2017-11-04-some-post.html`
```python
    @app.route('/blog/<year>/<month>/<day>/<name>.html')
    def blog_post(year, month, day, name):
        return send_from_directory('posts/', path + year + '-' + month + '-' + day + '-' + name + '.html')
```
- The path for this will be generated by the following freezer
```python
    @freezer.register_generator
    def blog_post():
        files = next(os.walk(BLOG_PATH))[2]
        blog_posts = [file for file in files if file.endswith('.html')]
        for post in blog_posts:
            path = split_path('', '', post)[0]
            yield {
                'year': path[0],
                'month': path[1],
                'day': path[2],
                'name': path[3][:-5],
            }
```

## Minification of CSS and JS
- The served JS and CSS should be minified (HTML too if possible)
- Use one of the Python libraries that minify files
- One of the example can be `pip install css-html-js-minify`
- Then just minifiy the css file in a couple of lines as follows
```python
    from css_html_js_minify import css_minify
    with open('main.css') as f:
        minified_result = css_minify(f.read())
```
- This can be added in the build pipeline and all the files can be minified

## Reduce the images using PIL
- We can further use the PIL library for processing images
- We can reduce the size of the images that are served
- Install `pip install Pillow`
```python
.
```

## Parallelizing some of the tasks
- We can parallelize some tasks that are done in loop
- Minifying files and processing images can be parallelize

In [9]:
import multiprocessing
from math import factorial
cpus = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cpus)
numbers = [36634, 25954, 32586, 27296, 36657, 25934, 32596, 27252, 36657, 25934, 32596, 27252]
%timeit list(map(factorial, numbers))
%timeit pool.map(factorial, numbers)

1 loop, best of 3: 229 ms per loop
10 loops, best of 3: 122 ms per loop


# Thank You